# Extract Eval Cases

Extract NL to code evaluation cases from the app modernization project. 

In [1]:
import json
import os

In [2]:
datasets = [
    "cinema_recommended",
    "coffee_shop_recommended",
    "company_1_recommended",
    "company_office_recommended",
    "county_public_safety_recommended",
    "customers_and_invoices_rcm_complex",
    "department_management_recommended",
    "museum_visit_recommended",
    "news_report_recommended",
    "northwind_one_to_one"
]


In [3]:
app_mod_dir = input("directory of the app mod project (e.g. /Users/<USER>/projects)")
print(f"App mod dir: {app_mod_dir}")
root_dir_path = os.path.join(app_mod_dir, "app-modernisation/tests/fixtures")

App mod dir: /Users/ben.p/projects


In [4]:
!pip install pyyaml
import yaml


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
from pprint import pprint
import re
test_cases_path = "test_cases"

dataset_and_test_cases_paths = list( map( lambda dataset: (dataset, os.path.join(root_dir_path, dataset, test_cases_path)), datasets))

def remove_num_prefix(s: str):
    """
    Removes the (#SOME_NUM.) Prefix from the case name as it exists in the yaml file.
    Ex: "#4. Show the cinema name..." --> Show the cinema name...
    """
    return re.sub(r'^#\d+\.\s*', '', s)
    
eval_cases = []
for (ds_name, path) in dataset_and_test_cases_paths:
    case_files = list(map(lambda f: os.path.join(path,f) , os.listdir(path)))
    for case_file in case_files:
        with open(case_file) as f:
            yaml_case = f.read()
            dict_case = yaml.safe_load(yaml_case)
            # pprint(dict_case)
            eval_case = {
                "input": {
                    "nl_query": remove_num_prefix(dict_case["name"]),
                    "dataset_name": ds_name,
                },
                "expected": dict_case["real_sql_output"],
                "metadata": {
                    "app_mod_case_id": int(os.path.splitext(os.path.basename(case_file))[0]),
                    "sql": {
                        "query": dict_case["payload"]["context"]["code"],
                        "tags": dict_case.get("tags", None)
                    }
                }
            }
            eval_cases.append(eval_case)

print(f"Num eval cases: {len(eval_cases)}")

Num eval cases: 177


In [7]:
data_out_dir = "data_out"
if not os.path.exists(data_out_dir):
    os.makedirs(data_out_dir)

path_out = os.path.join(data_out_dir,"eval_cases.json")
with open(path_out, "a") as f_out:
    f_out.write(json.dumps(eval_cases))
    print(f"Wrote eval cases to: {path_out}")
        

Wrote eval cases to: data_out/eval_cases.json
